# <center> Тестовое задание OSA Hybrid Platform </center>

"База данных имеет следующие реквизиты: хост - 188.93.23.29, порт - 5432, имя базы - test_db, логин пользователя - candidate, пароль - e2e4. Используемая СУБД - PostgreSQL 9.5.6. В качестве клиента можно использовать SQL Workbench/J (не то же самое, что MySQL Workbench), DBeaver, DataGrip, Navicat или PG Admin. При возникновении проблем с подключением можете писать нам.

Таблица osa_posdata_april_5_stores содержит чековые данные по 5 магазинам за апрель 2016 года. Таблица categories содержит данные о товарной иерархии, то есть о том, как товары структурированы по вложенным категориям. Таблица products содержит базовую информацию о каждом отдельном товаре, идентифицируемом по plu (Product Lookup Code). Назначение отдельных колонок этих таблиц и связи между ними нужно определить самостоятельно, проведя какие-либо эксперименты, основанные на здравом смысле и понимании того, что такое магазинные чеки.

Задача состоит из двух пунктов.

В первом требуется написать запрос, для каждого из дней апреля 2016 года возвращающий количество уникальных чеков, содержащих хотя бы один товар категории "Молоко" (id = FR0604000).

Во втором пункте требуется написать запрос, для каждого из покупателей, обладающих картой лояльности, возвращающий медианное значение интервалов между покупками, совершаемыми этим покупателем (датой покупки считается дата пробития чека и можно считать, что карта лояльности не передаётся другим людям)".


In [1]:
import pandas as pd
import psycopg2

In [2]:
def pg_query(host=None, query=None, as_df=True):    
    try:
        pgdb = psycopg2.connect(dbname="test_db", user="candidate", password="e2e4", host="188.93.23.29")        

        if as_df:
            df = pd.read_sql(query, pgdb)
            pgdb.close()
            return df
        else:
            pgcursor = pgdb.cursor()
            pgcursor.execute(query)
            data = [_ for _ in pgcursor.fetchall()]
            colnames = [desc[0] for desc in pgcursor.description]
            pgcursor.close()
            pgdb.close()
            return colnames, data

    except Exception as e:
        pgdb.close()
        print(e)
        return None, None
    finally:
        pgdb.close()

Поскольку по категории "Молоко" id = FR0604000 запрос в таблицу приходил пустой, сделано решения для молочных продуктов с id = FR060400* 

In [3]:
query1 = """
SELECT DATE(check_date_time), COUNT(check_number) AS Counts
FROM public.osa_posdata_april_5_stores
WHERE plu IN
    (SELECT plu FROM public.products
    WHERE category_id LIKE  'FR060400_')  /*ищем всю молочную продукцию */
GROUP BY DATE(check_date_time);
"""

In [4]:
df = pg_query(query=query1)
df[:10]

,date,counts
0,2016-04-01,4190
1,2016-04-23,4000
2,2016-04-15,4978
3,2016-04-30,3766
4,2016-04-16,4321
5,2016-04-11,4167
6,2016-04-29,4309
7,2016-04-14,4545
8,2016-04-26,4137
9,2016-04-13,3883


In [5]:
query2 = """
SELECT 
    loyalty_card,     
    percentile_cont(0.5) WITHIN GROUP (ORDER BY EXTRACT(DAY FROM (check_date_time))) AS  median_date_time
FROM public.osa_posdata_april_5_stores 
GROUP BY loyalty_card ;
"""

In [6]:
df = pg_query(query=query2)
df[:10]

,loyalty_card,median_date_time
0,7000246170,15.0
1,7003981617,6.0
2,7004061500,26.0
3,7008059559,30.0
4,7008979079,6.0
5,7008985530,25.0
6,7010210737,6.0
7,7020303175,28.0
8,7020426224,1.0
9,7020429137,12.0


In [7]:
del df